<a href="https://colab.research.google.com/github/dequiroz/1MTR53_RobIA/blob/main/2_DH_Manipulador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://drive.google.com/uc?export=view&id=1VBn7nKlruxCPLHH_UuD8B2pkvKP7JLRd" alt="drawing" width=800/>

<img src="https://drive.google.com/uc?export=view&id=1uF_4x1YqLuS3M0mlh3NrMrEUHvjBMBFw" alt="drawing" width=800/>

<img src="https://drive.google.com/uc?export=view&id=1vgFoFHEKMN0WcAFlKS2hPMg2KuBm3sp9" alt="drawing"/>

#Cinemática directa empleando Denavit-Hartenberg

##1. Introducción a transformación de Denavit-Hartenberg

Recordemos que los parámetros Denavit-Hartenberg nos ayudan a definir una serie de transformaciones entre sistemas de referencia

<img src="https://drive.google.com/uc?export=view&id=14p7kb9mt1v6ODLtkP1YDg1F_zzo-gC4q" alt="drawing" width="600"/>

Estos parámetros se pueden resumir en una matriz de transformación homogenea que representa las rotaciones y translaciones que se han realizado para desplazarse entre sistemas de referencia

<img src="https://drive.google.com/uc?export=view&id=1QsMcmOlMFuTW7e53alAqXUN_zg4Op4qU" alt="drawing" width="600"/>

##2. Análisis simbólico de transformación Denavit-Hartenberg

Para esté análisis se emplea la librería [sympy](https://www.sympy.org/en/index.html), la cual nos permitirá determinar las matrices de transformación homogeneas para cada eslabón

In [ ]:
import sympy as sp     # Se importa la biblioteca para el cálculo simbólico

In [ ]:
# =============================================================================
# Se definen una función para representar la transformación de Denavit-Hartenberg
# =============================================================================
def transformacion_DH(d: sp.symbols, a: sp.symbols, alfa: sp.symbols, theta: sp.symbols) -> sp.Matrix:
  """
  Retorna una representación simbólica de la transformación de Denavit-Hartenberg

  Parametros (sp.symbols):
    d    : Traslación 𝑑 a lo largo del eje Z
    a    : Traslación 𝑎 a lo largo del eje X
    theta: Rotación 𝜃 a lo largo del eje Z
    alfa : Rotación 𝛼 a lo largo del eje X
  """
  cos_theta = sp.cos(theta)
  sin_theta = sp.sin(theta)
  cos_alfa  = sp.cos(alfa)
  sin_alfa  = sp.sin(alfa)
  T = sp.Matrix([[cos_theta, -cos_alfa*sin_theta,  sin_alfa*sin_theta, a*cos_theta],
                  [sin_theta,  cos_alfa*cos_theta, -sin_alfa*cos_theta, a*sin_theta],
                  [0, sin_alfa, cos_alfa, d],
                  [0, 0, 0, 1]])
  return T

In [ ]:
# Para usar la función se debe definir las variables de forma simbólica que
# representan la geometría del eslabón
theta, d, a, alfa = sp.symbols("𝜃 d a 𝛼")

TDH = transformacion_DH(d, a, alfa, theta)
print("TDH:"); display(TDH)

TDH:


Matrix([
[cos(𝜃), -sin(𝜃)*cos(𝛼),  sin(𝛼)*sin(𝜃), a*cos(𝜃)],
[sin(𝜃),  cos(𝛼)*cos(𝜃), -sin(𝛼)*cos(𝜃), a*sin(𝜃)],
[     0,         sin(𝛼),         cos(𝛼),        d],
[     0,              0,              0,        1]])

## 3. Cálculo numérico

El mismo procedimiento se puede realizar de forma numérica empleando numpy para el manejo de matrices y vectores en python

In [ ]:
import numpy as np

In [ ]:
# =============================================================================
# Se definen una función para calcular la transformación de Denavit-Hartenberg
# =============================================================================
def transformacion_DH_num(d: float, a: float, alfa: float, theta: float) -> np.ndarray:
  """
  Retorna una representación numérica de la transformación de Denavit-Hartenberg

  Parametros (float):
    d    : Traslación 𝑑 a lo largo del eje Z
    a    : Traslación 𝑎 a lo largo del eje X
    theta: Rotación 𝜃 a lo largo del eje Z
    alfa : Rotación 𝛼 a lo largo del eje X
  """
  cos_theta = np.cos(theta)
  sin_theta = np.sin(theta)
  cos_alfa  = np.cos(alfa)
  sin_alfa  = np.sin(alfa)
  T = np.array([[cos_theta, -cos_alfa*sin_theta,  sin_alfa*sin_theta, a*cos_theta],
                [sin_theta,  cos_alfa*cos_theta, -sin_alfa*cos_theta, a*sin_theta],
                [0, sin_alfa, cos_alfa, d],
                [0, 0, 0, 1]])
  return T

In [ ]:
# Para usar la función se debe definir los valores correspondientes a los
# parámetros de Denavit-Hartenberg. Estos dependen de la geometría del robot
theta, d, a, alfa = (np.deg2rad(30), 10, 5, np.deg2rad(45))

TDH = transformacion_DH_num(d, a, alfa, theta)
print("TDH:")
print(np.round(TDH,3))

TDH:
[[ 0.866 -0.354  0.354  4.33 ]
 [ 0.5    0.612 -0.612  2.5  ]
 [ 0.     0.707  0.707 10.   ]
 [ 0.     0.     0.     1.   ]]


## 1. Robot manipulador de 2 grados rotacionales (RR)



Recordar el manipulador descrito

<p align="center">
<img src="https://drive.google.com/uc?export=view&id=1Gt7FztKupxVbmHPmauYFbJkDMeNEU9VO" alt="drawing" width="300"/></p>

del DCL del manipulador, se puede observar que el sistema de referencia 1 presenta las siguientes transformaciones respecto a la base del robot:

*   Rotación en el eje $Z$ de ángulo $\theta_1$
*   Traslación alrededor del eje $X_1$ de valor $l_1$

De igual forma, el sistema de referencia 2 presenta las siguientes transformaciones respecto al sistema de referencia 1:

*   Rotación en el eje $Z_1$ de ángulo $\theta_2$
*   Traslación alrededor del eje $X_1$ de valor $l_2$

Estas transformaciones se pueden expresar según la siguiente tabla de parámetros Denavit-Hartenberg

</br>

**Taba de DH: Robot RR:**   

|d|a|alpha|theta|
|-|-|-|-|
|0|$l_1$| 0|$q_1$|
|0|$l_2$| 0|$q_2$|


### Análisis simbólico

In [ ]:
# Definición de variables representativas de la geometría del manipulador
q1, q2, l1, l2 = sp.symbols("q1 q2 l1 l2")

# Se definen las matrices de transformacion homogenea correspondientes empleando
# la notación:
#    Tab: Transformación del sistema 'b' con respecto al sistema 'a'

# Se definen la transformación del sistema 1 (codo) respecto al sistema 0 (base del robot)
T01 = transformacion_DH(0,l1,0,q1)
# Se definen la transformación del sistema 2 (efector final) respecto al sistema 1 (codo)
T12 = transformacion_DH(0,l2,0,q2)
# Se definen la transformación del sistema 2 (efector final) respecto al sistema 0 (base del robot)
T02 = sp.simplify(T01*T12)

# Mostrar las transformaciones homogéneas (display funciona con IPython)
print("T01:"); display(T01)
print("T12:"); display(T12)
print("T02:"); display(T02)

T01:


Matrix([
[cos(q1), -sin(q1), 0, l1*cos(q1)],
[sin(q1),  cos(q1), 0, l1*sin(q1)],
[      0,        0, 1,          0],
[      0,        0, 0,          1]])

T12:


Matrix([
[cos(q2), -sin(q2), 0, l2*cos(q2)],
[sin(q2),  cos(q2), 0, l2*sin(q2)],
[      0,        0, 1,          0],
[      0,        0, 0,          1]])

T02:


Matrix([
[cos(q1 + q2), -sin(q1 + q2), 0, l1*cos(q1) + l2*cos(q1 + q2)],
[sin(q1 + q2),  cos(q1 + q2), 0, l1*sin(q1) + l2*sin(q1 + q2)],
[           0,             0, 1,                            0],
[           0,             0, 0,                            1]])

Analizando los resultados se puede observar que las matrices de transformaciones son las mismas que las obtenidas mediante el método geométrico

### Análisis numérico

In [ ]:
# =============================================================================
# Se define la función para calcular la cinemática directa del manipulador de
# 2 grados de libertad RR
# =============================================================================
def cinematica_directa_robot2D(q1: float, q2: float, l1: float, l2: float):
  """
  Calcula la cinemática directa del robot 2D

  Parámetros (float):
      q1: Ángulo de la articulación 1 en radianes.
      q2: Ángulo de la articulación 2 en radianes.
      l1: Longitud del eslabón 1.
      l2: Longitud del eslabón 2.

  Retorna:
      Una tupla de matriz 4x4 que representan las transformaciones homogénea de
      cada eslabón con respecto a la base del robot.
  """
  # Se definen las matrices de transformacion homogenea correspondientes empleando
  # la notación:
  #    Tab: Transformación del sistema 'b' con respecto al sistema 'a'
  T01 = transformacion_DH_num(0,l1,0,q1)
  T12 = transformacion_DH_num(0,l2,0,q2)
  T02 = T01 @ T12
  return T01, T02


In [ ]:
from ipywidgets import interact

# =============================================================================
# Se emplea la función interact (funciona en Jupiter o Colab) para generar un
# sistema interactivo.
# =============================================================================
@interact(q1=(-180, 180, 1), q2=(-180, 180, 1), l1=(0, 10, 1), l2=(0, 10, 1))
def cinematica_directa_interactivo(q1, q2, l1, l2):
  # Aplicación de cinemática directa numérica
  q1 = np.deg2rad(q1)
  q2 = np.deg2rad(q2)
  T01, T02 = cinematica_directa_robot2D(q1, q2, l1, l2)

  # Mostrar el resultado
  print(np.round(T02,3))

interactive(children=(IntSlider(value=0, description='q1', max=180, min=-180), IntSlider(value=0, description=…

### 3. Simulación gráfica

En esta sección se emplea la librería [matplotlib](https://matplotlib.org/) para graficar un esquema del manipulador

In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt

# =============================================================================
# Se emplea la función interact (funciona en Jupiter o Colab) para generar un
# sistema interactivo.
# =============================================================================
@interact(q1=(-180, 180, 1), q2=(-180, 180, 1), l1=(0, 10, 1), l2=(0, 10, 1))
def plot_robot(q1, q2, l1, l2):
  # Aplicación de cinemática directa numérica
  q1 = np.deg2rad(q1)
  q2 = np.deg2rad(q2)
  T01, T02 = cinematica_directa_robot2D(q1, q2, l1, l2)

  # Obtener las coordenadas del efector final
  x2 = T02[0, 3]
  y2 = T02[1, 3]

  # Obtener las coordenadas del codo
  x1 = T01[0, 3]
  y1 = T01[1, 3]

  # Graficar el robot
  plt.figure(figsize=(4, 4))
  plt.plot([0, x1, x2], [0, y1, y2], 'o-', color='purple', linewidth=2)
  plt.plot(0,0, x1, y1, marker='o', color='y', markersize=5)
  plt.xlabel('X')
  plt.ylabel('Y')
  plt.title('Robot de 2 grados de libertad RR')
  plt.xlim([-15, 15])
  plt.ylim([-15, 15])
  plt.grid(True)
  plt.show()

interactive(children=(IntSlider(value=0, description='q1', max=180, min=-180), IntSlider(value=0, description=…

### 4. Simulación gráfica 3D empleando visual_kinematics

En esta sección se emplea la librería [visual_kinematics](https://github.com/dbddqy/visual_kinematics) para graficar un esquema del manipulador en un entorno tridimensional.

Está librería no es estandard en colab por lo que se requiere instalar antes de poder usarla. **Ejecutar la siguiente linea para la instalación**

In [ ]:
!pip install visual_kinematics

La librería requiere la definición de los parámetros de DH en un arreglo de numpy como se muestra en el siguiente código

**Taba de DH: Robot RR:**   

|d|a|alpha|theta|
|-|-|-|-|
|0|$l_1$| 0|$q_1$|
|0|$l_2$| 0|$q_2$|

In [ ]:
from visual_kinematics.RobotSerial import *

@interact(q1=(-180, 180, 1), q2=(-180, 180, 1), l1=(0, 10, 1), l2=(0, 10, 1))
def plot_robot(q1, q2, l1, l2):
  # Definición se los parámetros Denavit-Hartenberg
  dh_params = np.array([[0, l1, 0, np.deg2rad(q1)],
                        [0, l2, 0, np.deg2rad(q2)]])
  # Se genera un brazo robot virtual a partir de los parámetros DH
  robot = RobotSerial(dh_params, plot_xlim=[-15, 15], plot_ylim=[-15, 15])
  robot.show()

interactive(children=(IntSlider(value=0, description='q1', max=180, min=-180), IntSlider(value=0, description=…

Prueba de cambios - github